In [2]:
x = torch.randn(2, 3)
torch.cat((x, x, x), -1)

tensor([[ 1.6311, -0.9878,  1.5576,  1.6311, -0.9878,  1.5576,  1.6311, -0.9878,
          1.5576],
        [ 0.8052, -0.3533,  0.6524,  0.8052, -0.3533,  0.6524,  0.8052, -0.3533,
          0.6524]])